In [1]:
from google.colab import drive
drive.mount('/content/drive')

# https://drive.google.com/drive/folders/1Tj1rGwyztX2MRlV1DkeBAPTbtoC9_-84?usp=share_link

Mounted at /content/drive


In [2]:
%%capture
# clone the repo
%cd /content/drive/MyDrive
%mkdir -p cs753-hacker
%cd cs753-hacker
!git clone https://github.com/facebookresearch/AVID-CMA

# get the checkpoint
%cd AVID-CMA
%mkdir -p checkpoints/AVID/Kinetics/Cross-N1024
!wget https://dl.fbaipublicfiles.com/avid-cma/checkpoints/AVID_Kinetics_Cross-N1024_checkpoint.pth.tar
!mv AVID_Kinetics_Cross-N1024_checkpoint.pth.tar checkpoint.pth.tar
!mv checkpoint.pth.tar checkpoints/AVID/Kinetics/Cross-N1024
%cd ..

# get the dataset
!wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
!sudo apt install unrar
!unrar x hmdb51_org.rar
%mkdir -p Dataset
!rm -rf hmdb51_org.rar
%mv *.rar Dataset

# get the splits
!wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
!unrar x test_train_splits.rar
%mkdir -p Data
!mv Dataset Data
!mv testTrainMulti_7030_splits Splits
!mv Splits Data

import os
%cd Data/Dataset
for rarfile in os.listdir('.'):
  !unrar x $rarfile
!rm *.rar
%cd ../..

!pip install av
%cd AVID-CMA

# change view to reshape in metrics_utils.py line 24
# change paths in hmdb.py to '../Data/Dataset' and '../Data/Splits'

/content/drive/MyDrive
/content/drive/MyDrive/cs753-hacker
Cloning into 'AVID-CMA'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 94 (delta 20), reused 15 (delta 15), pack-reused 50
Unpacking objects: 100% (94/94), 16.57 MiB | 4.32 MiB/s, done.


In [20]:
!python eval-action-recg.py configs/benchmark/hmdb51/8at16-fold1.yaml configs/main/avid/kinetics/Cross-N1024.yaml

==============================   Config   ==============================
required_devices: 8
resume: False
no_test: False
test_only: False
debug: False
seed: 0
distributed: False
test_freq: 5
num_workers: 2
benchmark
  name: hmdb51-wucls-8at16
dataset
  name: hmdb51
  fold: 1
  batch_size: 32
  clip_duration: 0.5
  video_fps: 16.0
  crop_size: 200
  transform: msc+color
  min_area: 0.08
  color: [1.0, 1.0, 1.0, 0.2]
  train
    split: train-split{fold}
    mode: clip
    clips_per_video: 25
    use_augmentation: True
    use_shuffle: True
    drop_last: True
  test
    split: test-split{fold}
    mode: clip
    clips_per_video: 1
    use_augmentation: False
    use_shuffle: False
    drop_last: False
  test_dense
    split: test-split{fold}
    mode: video
    clips_per_video: 10
    use_augmentation: False
    use_shuffle: False
    drop_last: False
optimizer
  name: adam
  num_epochs: 10
  weight_decay: 0.02
  warmup_classifier: True
  lr
    name: multistep
    base_lr: 0.001
    ga